In [1]:
%matplotlib inline
from pylab import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import networkx as nx
from networkx.drawing.nx_agraph import write_dot, graphviz_layout

### This is a tutorial for creating a treatment train one unit process at a time. The aim is to demonstrate design capabilities in WaterTAP3 using the "treatment_train_design" module. The steps are:
#### 1. Look up unit process library
#### 2. Look up source and end use source water options
#### 3. Create a treatment train with a unit process, defining the source and end use
#### 4. Add another unit process in parallel or series
#### 5. Add a recycle stream
#### 6. Add multiple water sources
#### 7. Add multiple end use water source
#### 8. Add a stream from any point to any point
#### 9. Remove unit process
#### 10. Save and load treatment train

##### Import WaterTAP3 Package

In [2]:
import watertap as wt

##### Step 1: Look up unit process library. Returns a List.

In [3]:
wt.unit_process_library_list

['chlorination_twb',
 'media_filtration_twb',
 'microfiltration_twb',
 'ultrafiltration_twb',
 'nanofiltration_twb',
 'ro_twb',
 'uv_twb',
 'ro_bor',
 'uvozone_twb',
 'mbr']

##### Step 2: Look up source and end use source water options. Returns a Pandas dataframe.

In [4]:
# source node name, feedwater column not used. TODO.
wt.water_source_use_library.head()

,Variable,Unit,Value,Reference,WaterType,CaseStudy,SourceOrUse,feedwater,SourceNodeName
Variable,,,,,,,,,
Turb,Turb,NTU,100.0,Poseidon,Wastewater,Typical untreated domestic wastewater,Source,100.0,source_node
TSS,TSS,mg/L,210.0,Poseidon,Wastewater,Typical untreated domestic wastewater,Source,210.0,source_node
BOD,BOD,mg/L,190.0,Poseidon,Wastewater,Typical untreated domestic wastewater,Source,190.0,source_node
COD,COD,mg/L,430.0,Poseidon,Wastewater,Typical untreated domestic wastewater,Source,430.0,source_node
TN,TN,mg/L,40.0,Poseidon,Wastewater,Typical untreated domestic wastewater,Source,40.0,source_node


##### Step 3: Create a treatment train with a unit process, defining the source and end use

In [5]:
# unit_process_name_list needs to be a list
unit_process_name_list = ['microfiltration_twb']
T = wt.treatment_train_design.create_train(unit_process_name_list=unit_process_name_list,
                                            source_water_reference = 'Poseidon',
                                            source_water_type = 'Wastewater',
                                            source_water_case_study ='Typical untreated domestic wastewater',
                                            enduse_water_reference = 'Poseidon',
                                            enduse_water_type = 'Environmental',
                                            enduse_water_case_study = 'EPA: Environmental Reuse',
                                            flow = 0)

adding unit process to network: microfiltration_twb
adding unit process attributes: microfiltration_twb


In [6]:
# display command
wt.display.show_train2(T)

##### Step 4: Add another unit process in parallel or series

In [91]:
##### ADD UNIT PROCESS IN PARALLEL ####
T = wt.treatment_train_design.add_unit_process(T, from_node = 'source',
                                                to_node = 'use', 
                                                unit_process_name_list = ['ultrafiltration_twb'],
                                                unit_process_list = ['ultrafiltration_twb'],
                                                order = 'parallel')

adding unit process to network: ultrafiltration_twb
adding unit process attributes: microfiltration_twb
adding unit process attributes: ultrafiltration_twb


In [92]:
wt.display.show_train2(T)

In [93]:
##### ADD UNIT PROCESS IN SERIES ####
T = wt.treatment_train_design.add_unit_process(T, from_node = 'use',
                                                to_node = None, 
                                                unit_process_name_list = ['chlorination_twb'],
                                                unit_process_list = ['chlorination_twb'],
                                                order = 'series')

adding unit process to network: chlorination_twb
adding unit process attributes: microfiltration_twb
adding unit process attributes: ultrafiltration_twb
adding unit process attributes: chlorination_twb


In [94]:
wt.display.show_train(T)

##### 5. Add a recycle stream

In [83]:
T = wt.treatment_train_design.add_recycle_stream(T,
                       unit_process = 'microfiltration_twb',
                       unit_process_name = 'microfiltration_twb',
                       recyle_fraction_of_waste = 1,
                       number_of_affected_streams = 1)

updating unit process attributes: microfiltration_twb


In [84]:
wt.display.show_train(T)

##### Step 6: Add multiple water sources

In [95]:
# ADD ANOTHER SOURCE. NEED TO PROVIDE A NEW SOURCE NAME.
source_to_node_names = [['source1', 'ultrafiltration_twb_start']]
T = wt.treatment_train_design.add_multiple_water_sources(T,
                                              number_of_additional_sources = 1,
                                              source_water_reference = 'Poseidon',
                                              source_water_type = 'Wastewater',
                                              source_water_case_study = 'Typical untreated domestic wastewater',
                                              source_to_node_names=source_to_node_names,
                                              flow = 0)

adding another water source: source1


In [96]:
wt.display.show_train(T)

##### Step 7: Add multiple end use water source

In [97]:
# ADD ANOTHER END USE 
node_to_use_names = [['chlorination_twb_end', 'use2']]
T = wt.treatment_train_design.add_multiple_water_enduses(T,
                                              number_of_additional_ends = 1,
                                              enduse_water_reference = 'Poseidon',
                                              enduse_water_type = 'Wastewater',
                                              enduse_water_case_study = 'Typical untreated domestic wastewater',
                                              node_to_end_names= node_to_use_names,
                                              min_flow = 0)

adding another water use: use2


In [98]:
wt.display.show_train(T)

##### Step 8: Add a stream from any point to any point. For this, user must define the type of stream being added. Options are: feedwater_stream, waste_stream, use_stream, recycle_stream, source_and_recycle_stream, treatment_process

In [99]:
for edge in T.edges():
    print(T.edges[edge]['name'], '----->>>' , T.edges[edge]['type'])

microfiltration_twb_feedwater ----->>> feedwater_stream
ultrafiltration_twb_feedwater ----->>> feedwater_stream
microfiltration_twb ----->>> treatment_process
microfiltration_twb_waste_stream ----->>> waste_stream
microfiltration_twb_use_stream ----->>> use_stream
chlorination_twb_feedwater ----->>> feedwater_stream
ultrafiltration_twb ----->>> treatment_process
ultrafiltration_twb_waste_stream ----->>> waste_stream
ultrafiltration_twb_use_stream ----->>> use_stream
chlorination_twb ----->>> treatment_process
chlorination_twb_waste_stream ----->>> waste_stream
chlorination_twb_use_stream ----->>> use_stream
chlorination_twb_end_treated_use2_stream ----->>> use_stream
source1_ultrafiltration_twb_start_feedwater ----->>> feedwater_stream


In [100]:
T = wt.treatment_train_design.add_stream(T, 
                                         from_node = 'microfiltration_twb_end', 
                                        to_node = 'use2',
                                        stream_name = 'treated_stream_example',
                                        stream_type = 'treated_stream') # TODO

In [101]:
wt.display.show_train(T)

##### Step 9: Remove unit process

In [20]:
T = wt.filter_processes.remove_single_process(T, process_name = 'ultrafiltration_twb')

In [102]:
wt.display.show_train(T)

##### Step 10: Save and load treatment train

In [103]:
#### SAVE TRAIN ####
path = 'Tutorial1_treatment_train_example.csv'
wt.save_train(T, path)

In [104]:
#### LOAD TRAIN ####
path = 'Tutorial1_treatment_train_example.csv'
TT = wt.load_train(path)

In [105]:
wt.display.show_train(TT)